# 基础概念

## 商品期货

![商品期货列表](images\商品期货列表.png)

In [1]:
# coding=utf-8
import sys
sys.path.append("C:\\Users\\hushuntai\\svn\\python")
import datetime as dt

import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib.dates as mdate
%matplotlib inline
from tqdm import tqdm_notebook

import QuantStudio.api as QS

In [7]:
# 参数设置和数据准备
WDB = QS.FactorDB.WindDB2()
WDB.connect()

FutureCodes = WDB.getFutureCode(exchange=["SHFE", "INE", "DCE", "CZCE"], is_current=True)
FutureIDs = WDB.getFutureID(future_code=FutureCodes, is_current=False, contract_type="月合约")
FT = WDB.getTable("中国期货基本资料")

Today = dt.datetime.combine(dt.date.today(), dt.time(0))
FutureInfo = FT.readData(factor_names=["标准合约代码", "证券中文简称", "交易所", "上市日期"], ids=FutureIDs, dts=[Today]).iloc[:, 0]
WDB.disconnect()

FutureInfo = FutureInfo.groupby(by=["标准合约代码"]).min()
FutureInfo["证券中文简称"] = FutureInfo["证券中文简称"].str.replace("\d", "")
FutureInfo = FutureInfo.sort_values(by=["交易所", "上市日期"])
display(FutureInfo)

,证券中文简称,交易所,上市日期
标准合约代码,,,
CF,郑棉,CZCE,20040601
SR,白糖,CZCE,20060106
TA,PTA,CZCE,20061218
PM,普麦,CZCE,20120117
OI,菜油,CZCE,20120716
RI,早籼稻,CZCE,20120724
WH,强麦,CZCE,20120724
FG,玻璃,CZCE,20121203
RM,菜粕,CZCE,20121228


## CTA

商品交易顾问(Commodity trading advisors, CTA)基金, 也称作管理期货(Managed futures), 它是指由专业的资金管理人运用客户委托的资金自主决定投
资于全球期货和期权市场以获利的并且收取相应管理费用的一种基金组织形式. 由于其高收益、低回撤以及和股票市场低相关性等优势, 目前 CTA 基金行业已经成为了全球发展最快的投资领域之一.

# 能源化工系期货

能源化工产业的产生与发展，基本上来源于对地球上不可再生资源（主要是石油、天然气及煤）的开采及后续利用。从产业链的角度来看，能源化工的产业链比较长，其涉及到的上、中、下游的细分行业众多，而且不同细分行业所受到的影响因素也较为复杂。综合国内的能源化工产业来看，其大致可以分为两大领域:
* 石油化工产业: 主要是指以石油及天然气为原料生产化学工业制品的产业
* 煤化工产业: 主要是一些以煤为原料并经过化学加工使煤转化为其他化学品的工业产业

这两大类产业的工业产值合起来大约占全国工业总产值的 20%（ 2011 年数据统计）

国际市场上，更为投资者所关注的是原油期货的价格，包括美国西德克萨斯出产并在纽约商业交易所（ NYMEX）上市的轻质原油（ WTI 原油）和出产于北大西洋北海布伦特地区并在伦敦洲际交易所上市的轻质原油（ Brent 原油）。从供给端来看，原油开采加工业属于能源化工期货相关的上游产业，原油的供给量会很大程度上影响甚至决定未来一段时期内其他能源化工期货价格的走势。所以，分析国际市场上原油期货价格的变动的原因能够帮助分析预测国内相关期货品种的未来走势。

从需求端来看，由于能源化工产业链呈现上窄下宽的“金字塔式”特点，产业越靠近上端，相关产品的生产集中度越高，价格垄断能力也就越高。另一方面，能源化工产业链下游的相关商品种类众多，所涉及的细分行业也相对广泛，因此对需求造成的不确定性较为明显，进而使得交易者对期货的未来价格走势也会做出不同的预期。

# 黑色系期货

黑色系期货主要是一般是指跟黑色金属——铁、铬、锰相关的一些商品期货，比如铁矿石、螺纹钢以及和钢铁冶炼关系密切的焦煤、焦炭.

美国 NYMEX 上市的 HRC 钢卷以及伦敦金属交易所上市的 LME 钢坯由于交易量特别少而几乎不受关注。

根据黑色系相关商品的特点来看，该产业链的上下游产品之间具有相比于其他产业更为紧密的关系，因此黑色系期货不同品种的价格之间也就表现出特别显著的强相关性。

从黑色系产业的供给端来看，铁矿石和焦煤焦炭是生产钢材的两类重要原材料。我国作为世界上最大的铁矿石消费国，国内铁矿石产量已不足以满足当前经济发展的需求，所以也使得我国成为最大的铁矿石进口国。从成本管理的角度考虑，铁矿石期货的价格不仅仅会受到开采时人工、电费、设备等因素的影响，而且也会受到国际海运、关税等因素的影响。焦煤焦炭等一系列冶炼钢铁原材料的生产成本也同时对钢材期货的价格造成影响。从此之外，煤矿、铁矿的开采以及钢材的生产加工往往都会受到政府出台的钢铁煤炭产业政策的影响，如果政策性成本上升，势必会对相关产品的供给带来压力，从而造成期货价格的变化。

从需求端来看，对黑色系期货价格影响最大的是宏观经济环境形势的变化。如果国民经济处于上升阶段，基础设施建设和房地产行业会刺激并拉动钢材或其他下游行业产品的需求，从而造成供不应求的局面，并在一定时期内使得相应商品期货的价格上涨；而如果经济处于下行，会造成钢材等产品库存量的增加，从而导致相应期货价格下跌。

# 有色系期货

有色系期货主要是指以有色金属为交割标的物的期货合约。而有色金属是指除了黑色金属以外的其他金属，包括贵金属（黄金、白银等）以及一般金属（铜、铝、铅、锌、锡、镍等）。国外期货市场交易上市的有色期货品种主要是 COMEX 的黄金、白银和 LME 的其他一般有色金属。

有色金属中的黄金、白银属于贵金属，由于其所具有的化学稳定性以及稀有特殊性，使得黄金、白银从很早的时候就具备了流通货币、金融储备、价值衡量等属性。所以能够对黄金白银期货价格产生影响的主要因素是国际政治环境、经济形势的变化，这其中包括美元指数、 PMI 指数等相关的宏观指标。

国内上期所的黄金期货价格也基本上与国际黄金期货市场上的价格走势保持一致。由于大多数有色金属价格在国际市场上是以美元标价，所以一般情况下，美元指数上涨会使得以美元计价的商品的价格下跌，而美元贬值时又会推动这些商品的价格上涨，它们与美元指数之间基本上保持比较明显的“此消彼长”的负相关关系。

由于有色金属相较于其他的期货品种具有较高的流动性，而且我国的有色金属矿产自给率普遍不足，从长期来看，国内市场在供给端和需求端会受到全球市场的影响，从而导致了国内的有色金属期货价格与国外对应品种的期货价格走势基本一致。

从供给端来看，有色金属属于矿产资源，市场的供给来源主要是国内外相应矿产的开采利用，所以开采设备、人力、水电费等构成了开采供给中不可避免的成本费用。由于有色金属在自然界大都是以混合物的形式存在的，所以在开采之后还需要一系列的金属冶炼过程，而在工业上不同金属的冶炼方式也是不尽相同的。其中很多都会涉及到烧结、电解的过程，所以煤炭和电的价格也会对有色金属冶炼企业的生产造成较大的影响。除此之外，国际市场上有色金属的价格波动也会影响到相应品种的进出口，同时反过来又会对推动国内市场上有色金属期货价格产生波动。

有色金属可应用的范围非常广，涉及到国民经济发展建设、人民日常生活以及国防工业等方方面面，甚至有的国家已经把有色金属作为重要的战略资源并增加储备。从具体的有色金属下游行业来看，工业制造业、建筑业、交通运输业及一般居民消费品行业对有色金属的需求占比还是很高的，所以下游行业发展的景气程度，决定了对有色金属的需求程度。从政治、经济局势来看，如果政策向好、经济上行，将会拉动有色金属行业的发展，进而提高对期货价格的未来预期。

# 农产品系期货

农产品系期货是指和农业产业生产品相关的期货合约，包括粮食类期货、 经济作物类期货等。我国是农业生产和消费大国，农业在国民经济中处于最基本的地位，直接关系到其他产业的发展，以及国民日常生活的切身利益和社会的安定。在我国期货市场上市交易的农产品期货合约主要是由大连商品期货交易所和郑州商品期货交易所推出的，而国外上市交易的主要农产品期货主要是由芝加哥商业交易所和纽约期货交易所推出的。

整体来看，农产品的生产具有严格的周期性和季节性特点，每一种农产品从种植到收获都有其固定的周期长度，这就制约了农产品的供给，使得其呈现一定的周期性特点。在农产品的种植生产过程中，还会受到一个很重要的因素——天气情况的制约。它们会对温度、光照、雨水等天气条件非常敏感，以至于稍微有一点的改变，就有可能对农产品的生产产生不利的影响，进而影响到农产品的供给量。在农产品生产过程中农机农药的使用，直接决定了相关农作物的生长情况。

在分析农产品价格变动因素的时候，我们首先会需要考虑农产品的主产地的产量情况，因为主产地的产量基本决定了全球范围内农产品的供给情况。由于我国的人口众多，对一些农产品的需求旺盛，甚至仍然需要大量依赖于进口。因此同样需要知道不同农产品的自给率（当年产量/表观消费量）以及对外依存度（进口量/表观消费量）的情况.

但是对于作为经济作物的大豆来说，虽然我国产量排名靠前，但大豆的自给率却远远不足，更多地需要从美国、巴西等国家进口，因此国内期货价格会受到主产国的供给情况以及进出口情况等方面的影响。实际上不仅仅大豆，大豆的相关品种豆油、豆粕也因此会受到国外市场价格波动的影响，其他的经
济作物类期货还有白糖、棉花、棕榈油，这一些农产品的自给率相比来说也是不足。甘蔗是用来生产白糖的主要原料，而我国的甘蔗种植量已经不能满足国内对白糖的消费需求，而且我国对棕榈油的供应也完全依赖于进口，所以对于白糖期货、棕榈油期货来说，它们的价格走势更多地会受到国际市场上期货价格走势的影响。我国的棉花除了供给国内市场，还有很大产量用于出口海外市场。国际市场上的供需变化、价格水平以及进出口政策会对国内的棉花期货价格造成直接的影响。

大豆的主要用途之一是用来压榨制作豆油，而油菜籽是另一种油料作物，主要是用来压榨生产菜籽油。所以作为其替代品，油菜籽以及其他油料作物花生、棉籽的产量、价格及消费的变化也会间接地影响到大豆类、油菜籽类期货价格的走势。除了豆油和菜籽油之外，大豆和油菜籽的其他下游产品分别为豆粕和菜籽粕，它们的需求量变化也会直接影响到大豆、油菜籽的需求，从而反映到期货价格上面。